In [2]:
import os
import urllib
import zipfile

# Download and unzip the dataset
if not os.path.isfile('plantdoc.zip'):
  urllib.request.urlretrieve("https://github.com/Mouneshgouda/PlantDoc-Dataset/archive/refs/heads/master.zip", "plantdoc.zip")

zip_filename = "plantdoc.zip"
with zipfile.ZipFile(zip_filename, "r") as zip_ref:
    zip_ref.extractall(".")


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Config
IMG_SIZE = 64
BATCH_SIZE = 32

# Dataset Paths (based on your GitHub repo)
train_path = "/content/PlantDoc-Dataset-master/train"
val_path = "/content/PlantDoc-Dataset-master/test"

# Data Loaders
train_data = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_data = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# MLP Model
model = Sequential([
    Flatten(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_data, epochs=5, validation_data=val_data)


Found 2342 images belonging to 28 classes.
Found 236 images belonging to 27 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 588ms/step - accuracy: 0.0416 - loss: -11110.1562

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


74/74 ━━━━━━━━━━━━━━━━━━━━ 49s 653ms/step - accuracy: 0.0415 - loss: -11319.4453 - val_accuracy: 0.0381 - val_loss: -78855.7109
Epoch 2/5
74/74 ━━━━━━━━━━━━━━━━━━━━ 45s 611ms/step - accuracy: 0.0349 - loss: -102923.4922 - val_accuracy: 0.0381 - val_loss: -280795.9375
Epoch 3/5
74/74 ━━━━━━━━━━━━━━━━━━━━ 45s 608ms/step - accuracy: 0.0350 - loss: -317749.5625 - val_accuracy: 0.0381 - val_loss: -617009.8125
Epoch 4/5
74/74 ━━━━━━━━━━━━━━━━━━━━ 46s 618ms/step - accuracy: 0.0398 - loss: -641105.3125 - val_accuracy: 0.0381 - val_loss: -1094269.0000
Epoch 5/5
74/74 ━━━━━━━━━━━━━━━━━━━━ 45s 611ms/step - accuracy: 0.0328 - loss: -1095187.1250 - val_accuracy: 0.0381 - val_loss: -1708770.2500


In [5]:
# Step 1: Download dataset ZIP from URL
!wget https://github.com/Mouneshgouda/PlantDoc-Dataset/archive/refs/heads/master.zip -O plantdoc.zip

# Step 2: Unzip the dataset
import zipfile
with zipfile.ZipFile("plantdoc.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

# Define the dataset directory path
data_dir = "/content/PlantDoc-Dataset-master"

# Step 3: Import required libraries
import numpy as np
import os
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from google.colab import files

# Step 4: Prepare data
img_size = (128, 128)

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Step 5: Build the ANN model for binary classification
model = Sequential([
    Flatten(input_shape=(128, 128, 3)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_gen, validation_data=val_gen, epochs=5)

# Step 6: Save the trained model
model.save("plant_ann_model.h5")

# Step 7: Upload and predict a single image
uploaded = files.upload()  # Upload an image from your PC

# Get the uploaded image path
img_path = next(iter(uploaded))

# Load and preprocess the image
img = load_img(img_path, target_size=img_size)
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Load the model and predict
model = load_model("plant_ann_model.h5")
prob = model.predict(img_array)[0][0]
label = "Disease found" if prob > 0.5 else "No disease found"
print(f"Predicted class: {label} (Confidence: {prob * 100:.2f}%)")


--2025-07-10 01:33:26--  https://github.com/Mouneshgouda/PlantDoc-Dataset/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Mouneshgouda/PlantDoc-Dataset/zip/refs/heads/master [following]
--2025-07-10 01:33:26--  https://codeload.github.com/Mouneshgouda/PlantDoc-Dataset/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘plantdoc.zip’

plantdoc.zip            [                <=> ] 938.81M  25.8MB/s    in 34s     

2025-07-10 01:34:01 (27.5 MB/s) - ‘plantdoc.zip’ saved [984413268]

Found 2063 images belonging to 2 classes.
Found 515 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 639ms/step - accuracy: 0.8141 - loss: 2.9960

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


65/65 ━━━━━━━━━━━━━━━━━━━━ 56s 828ms/step - accuracy: 0.8147 - loss: 2.9729 - val_accuracy: 0.8757 - val_loss: 0.3698
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 51s 782ms/step - accuracy: 0.8811 - loss: 0.4932 - val_accuracy: 0.9087 - val_loss: 0.6148
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 51s 782ms/step - accuracy: 0.8962 - loss: 0.4092 - val_accuracy: 0.9087 - val_loss: 0.4574
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 50s 771ms/step - accuracy: 0.8992 - loss: 0.3571 - val_accuracy: 0.9010 - val_loss: 0.3341
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 82s 768ms/step - accuracy: 0.8993 - loss: 0.2907 - val_accuracy: 0.6330 - val_loss: 0.8002


Saving f2c48f6c-f92e-4543-816c-0621648003fe___JR_B.Spot 9010.JPG to f2c48f6c-f92e-4543-816c-0621648003fe___JR_B.Spot 9010.JPG
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted class: No disease found (Confidence: 40.59%)


In [ ]:
!wget https://github.com/pratikkayal/PlantDoc-Dataset/archive/refs/heads/master.zip -O plantdoc.zip

# Step 2: Unzip the dataset
import zipfile
with zipfile.ZipFile("plantdoc.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")